In [1]:
import pandas as pd
import numpy as np
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
train = pd.read_csv('clean_train.csv')
test = pd.read_csv('test.csv')
train['source'] = 'train'
test['source'] = 'test'
merged = pd.concat([train, test])

In [3]:
def getTrainSet():
    return merged.loc[merged['source'] == 'train']

def getTestSet():
    return merged.loc[merged['source'] == 'test']

### Agregando caracteristicas del texto

In [4]:
def getHashtags(words):
    return ' '.join([item.lstrip('#') for item in words if item.startswith('#') and len(item) > 1])

def getMentions(words):
    return ' '.join([item.lstrip('@') for item in words if item.startswith('@') and len(item) > 1])

def getURLs(words):
    return ' '.join([item for item in words if item.startswith('http')])

#import nltk
#nltk.download('stopwords')
#Elimina las stopwords, urls, hashtags y menciones
def removeSpecialWords(words):
    stop = stopwords.words('english')
    return ' '.join([item.lstrip('#').lstrip('@') for item in words if item not in stop and not item.startswith('#') and not item.startswith('@') and not item.startswith('http')])

import re
regex = re.compile('[^a-zA-Z]')
def getPunctuationSigns(words):
    return ' '.join([item for item in words if len(regex.sub('', item)) == 0])

def removePunctuationSigns(words):
    return [item for item in words if len(regex.sub('', item)) > 0]

def arrayToLower(arr):
    return [item.lower() for item in arr]

def getWordsLengthAVG(text):
    words = text.split()
    
    if len(words) == 0:
        return 0
    
    total = 0
    for w in words:
        total += len(w)
        
    return total/len(words)

In [5]:
tweet_tokenizer = TweetTokenizer()

merged['words'] = merged['text'].apply(tweet_tokenizer.tokenize).apply(arrayToLower) #Tokenizacion
merged['real_words'] = merged['words'].apply(removePunctuationSigns) #Elimino los signos de puntuacion
merged['clean_text'] = merged['real_words'].apply(removeSpecialWords) #Elimino las stopwords, hashtags, menciones y urls

merged['punctuation_signs'] = merged['words'].apply(getPunctuationSigns) #Obtengo los signos de puntuacion "s1 s2 s2 ..."
merged['hashtags'] = merged['words'].apply(getHashtags) #Obtengo los hashtags: "ht1 ht2 ht3 ..."
merged['mentions'] = merged['words'].apply(getMentions) #Obtengo las menciones: "men1 men2 men3 ..."
merged['urls'] = merged['words'].apply(getURLs) #Obtengo las urls "url1 url2 url3 ..."

In [6]:
merged['entities_count'] = merged['words'].apply(lambda x: len(x)) #Conteo de todas las cosas que hay (incluye TODO)
merged['words_count'] = merged['clean_text'].apply(lambda x: len(x.split())) #Conteo de palabras
merged['punctuations_signs_count'] = merged['punctuation_signs'].apply(lambda x: len(x.split())) #Conteo de puntuaciones
merged['hashtags_count'] = merged['hashtags'].apply(lambda x: len(x.split())) #Conteo de hashtags
merged['mentions_count'] = merged['mentions'].apply(lambda x: len(x.split())) #Conteo de menciones
merged['urls_count'] = merged['urls'].apply(lambda x: len(x.split())) #Conteo de urls

#Conteo de stopwords
merged['stopwords_count'] = merged.entities_count - merged.words_count - merged.punctuations_signs_count - merged.hashtags_count - merged.mentions_count - merged.urls_count

In [7]:
merged['words_length_avg'] = merged['clean_text'].apply(getWordsLengthAVG)

merged['punctuations_ratio'] = merged.punctuations_signs_count/merged.entities_count
merged['hashtags_ratio'] = merged.hashtags_count/merged.entities_count
merged['mentions_ratio'] = merged.mentions_count/merged.entities_count
merged['urls_ratio'] = merged.urls_count/merged.entities_count
merged['stopwords_ratio'] = merged.stopwords_count/merged.entities_count

merged['special_entities_ratio'] = (merged.entities_count - merged.words_count)/merged.entities_count

## Aplicando mean encoding a keyword

In [8]:
#Completando valores vacios
merged['keyword'].fillna('undefined', inplace=True)
kw_group = merged.groupby('keyword')['target'].mean().reset_index()
kw_group.columns = ['keyword', 'keyword_cv_mean_enc']

#Agrego la columna keyword_cv_mean_enc
merged = pd.merge(merged, kw_group, how='left', on='keyword')

In [9]:
merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10766 entries, 0 to 10765
Data columns (total 32 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   id                        10766 non-null  int64  
 1   keyword                   10766 non-null  object 
 2   location                  7186 non-null   object 
 3   text                      10766 non-null  object 
 4   target                    7503 non-null   float64
 5   country                   7503 non-null   object 
 6   city                      7503 non-null   object 
 7   lat                       4273 non-null   float64
 8   lon                       4273 non-null   float64
 9   source                    10766 non-null  object 
 10  words                     10766 non-null  object 
 11  real_words                10766 non-null  object 
 12  clean_text                10766 non-null  object 
 13  punctuation_signs         10766 non-null  object 
 14  hashta

In [10]:
merged.head()

,id,keyword,location,text,target,country,city,lat,lon,source,...,urls_count,stopwords_count,words_length_avg,punctuations_ratio,hashtags_ratio,mentions_ratio,urls_ratio,stopwords_ratio,special_entities_ratio,keyword_cv_mean_enc
0,1,undefined,NaN,Our Deeds are the Reason of this #earthquake M...,1.0,undefined,undefined,NaN,NaN,train,...,0,6,4.666667,0.000,0.076923,0.0,0.0,0.461538,0.538462,0.660714
1,4,undefined,NaN,Forest fire near La Ronge Sask. Canada,1.0,undefined,undefined,NaN,NaN,train,...,0,0,4.428571,0.125,0.000000,0.0,0.0,0.000000,0.125000,0.660714
2,5,undefined,NaN,All residents asked to 'shelter in place' are ...,1.0,undefined,undefined,NaN,NaN,train,...,0,11,7.090909,0.120,0.000000,0.0,0.0,0.440000,0.560000,0.660714
3,6,undefined,NaN,"13,000 people receive #wildfires evacuation or...",1.0,undefined,undefined,NaN,NaN,train,...,0,1,7.800000,0.125,0.125000,0.0,0.0,0.125000,0.375000,0.660714
4,7,undefined,NaN,Just got sent this photo from Ruby #Alaska as ...,1.0,undefined,undefined,NaN,NaN,train,...,0,7,4.571429,0.000,0.125000,0.0,0.0,0.437500,0.562500,0.660714


In [11]:
from nltk.stem import PorterStemmer
from nltk.stem import SnowballStemmer

from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

In [12]:
#Si tira error porque no lo reconoce ejecutar el siguiente codigo
#import nltk
#nltk.download('wordnet')

#Si tira error por no encontrar el 'averaged_perceptron_tagger' ejecutar el siguiente codigo
#nltk.download('averaged_perceptron_tagger')

def getWordNetPOSTag(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

def lemmatizer(text):
    lemmatizer = WordNetLemmatizer()
    lemmaWords = []
    for w in text.split(" "):
        #lemmaWords.append(lemmatizer.lemmatize(w, getWordNetPOSTag(w)))
        lemmaWords.append(lemmatizer.lemmatize(w, pos='a')) 
        
    return " ".join(lemmaWords)

def porterStemmer(text):
    porter = PorterStemmer()
    stemmedWords = []
    for w in text.split(" "):
        stemmedWords.append(porter.stem(w)) 
        
    return " ".join(stemmedWords)

def snowballStemmer(text):
    porter = SnowballStemmer("english")
    stemmedWords = []
    for w in text.split(" "):
        stemmedWords.append(porter.stem(w)) 
        
    return " ".join(stemmedWords)

In [13]:
merged['lemma_text'] = merged['clean_text'].apply(lemmatizer)
merged['porter_stemmed_text'] = merged['clean_text'].apply(porterStemmer)
merged['snowball_stemmed_text'] = merged['clean_text'].apply(snowballStemmer)

In [34]:
merged.to_csv('final_train.csv', index=False)

In [23]:
lemma_vector = TfidfVectorizer(sublinear_tf=True)
lemma_vector.fit(np.array(merged['lemma_text']).ravel())
#lemma_vector.transform(np.array(merged['lemma_text']).ravel())

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=True, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [30]:
lemma_vector = TfidfVectorizer(sublinear_tf=True)
lemma_vector.fit(np.array(getTrainSet()['clean_text']).ravel())
lemma_vector.transform(np.array(getTrainSet()['clean_text']).ravel())

<7503x13137 sparse matrix of type '<class 'numpy.float64'>'
	with 60588 stored elements in Compressed Sparse Row format>

In [23]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

def testColumn(columnName):
    feature_vector = TfidfVectorizer(sublinear_tf=True)
    feature_vector.fit(np.array(getTrainSet()[columnName]).ravel())

    tf_vector = feature_vector
    X = lemma_vector.transform(np.array(getTrainSet()[columnName]).ravel())
    y = np.array(train['target']).ravel()
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=30)

    # Training Naive Bayes model
    NB_model = MultinomialNB()
    NB_model.fit(X_train, y_train)
    y_predict_nb = NB_model.predict(X_test)
    print(f1_score(y_test, y_predict_nb))

    # Training Logistics Regression model
    LR_model = LogisticRegression(solver='lbfgs')
    LR_model.fit(X_train, y_train)
    y_predict_lr = LR_model.predict(X_test)
    print(f1_score(y_test, y_predict_lr))

In [31]:
testColumn('lemma_text')

0.741652021089631
0.7383512544802868


In [32]:
testColumn('clean_text')

0.7421052631578948
0.7394807520143241


In [33]:
testColumn('porter_stemmed_text')

0.7070524412296565
0.7314487632508834


In [ ]:
testColumn('snowball_stemmed_text')

In [ ]:
from sklearn.feature_selection import SelectKBest

X = getTrainSet()[['entities_count', 'words_count', 'punctuations_signs_count', 'hashtags_count', 'mentions_count', 'urls_count', 
           'stopwords_count', 'words_length_avg', 'punctuations_ratio', 'mentions_ratio', 'urls_ratio', 'stopwords_ratio',
           'special_entities_ratio', 'keyword_cv_mean_enc']].values
Y = getTrainSet()['target'].values
# feature extraction
test = SelectKBest(score_func=f_classif, k=4)
fit = test.fit(X, Y)
# summarize scores
set_printoptions(precision=3)
print(fit.scores_)
features = fit.transform(X)
# summarize selected features
print(features[0:5,:])

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier

X = getTrainSet()[['entities_count', 'words_count', 'punctuations_signs_count', 'hashtags_count', 'mentions_count', 'urls_count', 
           'stopwords_count', 'words_length_avg', 'punctuations_ratio', 'mentions_ratio', 'urls_ratio', 'stopwords_ratio',
           'special_entities_ratio', 'keyword_cv_mean_enc']].values
Y = getTrainSet()['target'].values
# feature extraction
model = ExtraTreesClassifier(n_estimators=10)
model.fit(X, Y)
print(model.feature_importances_)

In [27]:
feature_vector = TfidfVectorizer(sublinear_tf=True)
feature_vector.fit(np.array(getTrainSet()['lemma_text']).ravel())

tf_vector = feature_vector
X = lemma_vector.transform(np.array(getTrainSet()['lemma_text']).ravel())
y = np.array(train['target']).ravel()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=30)

X_to_predict = lemma_vector.transform(np.array(getTestSet()['lemma_text']).ravel())

NB_model = MultinomialNB()
NB_model.fit(X_train, y_train)

predictions = NB_model.predict(X_to_predict)

In [36]:
def generateSubmission(predicts, model):
    submission = test[['id', 'target']]
    submission['target'] = predicts
    submission.to_csv(model + '_pred.csv', index=False)

In [37]:
generateSubmission(predictions, 'NaiveBayes')

C:\ProgramData\Miniconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
